In [1]:
reload_lamb()

# Introduction to type shifting in ILNB
### Notebook author: Kyle Rawlins

This notebook goes through the basics of how to do type-shifting in the lambda notebook.  Basically, you add a unary rule to the composition system, designate it as a typeshift, and enable type-shifting.  Currently, type-shifting is purely as a last resort.  (If you need to simulate other cases, it is best right now just to use unary rules or extra lexical entries.)

In general, I highly recommend that type-shifting be implemented via *combinators*.  A combinator is simply any function that has no free variables.  To build a typeshifting combinator, simply take the typeshift and use the shifted type as a further lambda term.  This strategy will make your life much easier, as it relies only on functional application to do its work.  The alternative, which may be helpful in certain special cases (not documented here) would be to write python code that performs the shift.

For example, here is a standard way of defining a type-lifting rule from Partee:

 * lift: $x \rightarrow \lambda f_{<e,t>} : f(x)$
 
The combinator for this typeshift is:

In [2]:
%%lamb
gqlift = L x_e : L f_<e,t> : f(x)

${gqlift}_{\left\langle{}e,\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}\:=\:\lambda{} x_{e} \: . \: \lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({x})$

A combinator like this can be defined as a ILNB meta-language expression, and conveniently also can be called to perform the type-shift, like any python callable.

In [3]:
%lamb thecat = Iota x_e : Cat_<e,t>(x)
gqlift(thecat).reduce_all()

(λ f_<e,t>: f_<e,t>((ι x_e: Cat_<e,t>(x_e))))

(All of the combinators I discuss here are of fixed type.  For flexibly typed combinators, see the examples in the "Variable free binding" notebook.)

### Predicate Modification (PM via type-shifting

The normal PM rule is already implemented in the lambda notebook via a combinator.  This means it is a good candidate for a type-shift.

In [4]:
%%lamb
||gray|| = L x_e : Gray(x)
||cat|| = L x_e : Cat(x)

INFO (meta): Coerced guessed type for 'Gray_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Gray}({x})$<br />
$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$

In [5]:
(gray * cat).tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Gray}({x})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[gray cat]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: ({Gray}({x}) \wedge{} {Cat}({x}))$</div></div></td><td></td></tr></table><br /><br />

In [6]:
pm_combinator = te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")
pm_combinator

(λ f_<e,t>: (λ g_<e,t>: (λ x_e: (f_<e,t>(x_e) & g_<e,t>(x_e)))))

The following code creates a composition system with the PM-shift instead of the regular PM operation.

Note the neat trick whereby the combinator can be used directly as the typeshift function.  You won't always want to do this, but it is very clean when it works.  It works because though the combinator is a TypedExpr, `__call__` is defined and so it can be treated like any other python function with one argument (exactly what the factory is looking for).  The reduction step is applied by the object constructed from this factory, and can be disabled with a keyword argument.

In [7]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(pm_combinator, "PM-shift", typeshift=True)) # add this as a typeshift
system.remove_rule("PM") # remove ordinary PM
system.typeshift = True # enable typeshifts in the composition system
lang.set_system(system) # set the new system as the default
system

Composition system: H&K simple (copy)

In [8]:
r = (gray * cat)
r

CompositionResult(results=[⟦[[gray] cat]⟧ = (λ x_e: (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))), ⟦[[cat] gray]⟧ = (λ x_e: (Cat_<e,t>(x_e) & Gray_<e,t>(x_e)))], failures=[⟦[[gray] [cat]]⟧ = Type mismatch: '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[[cat] [gray]]⟧ = Type mismatch: '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[[gray] [cat]]⟧ = Type mismatch: '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[cat] [gray]]⟧ = Type mismatch: '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[cat [gray]]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[[gray] cat]⟧ = Type mismatch: '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[cat [gray]]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦[gray]⟧ = (λ g_<e,t>: (λ x_e: (Gray_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[gray [cat]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[gray [cat]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[cat] gray]⟧ = Type mismatch: '⟦[cat]⟧ = (λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))'/<<e,t>,<e,t>> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction)])

In [9]:
r.tree()

2 composition paths:<br />
Path [0]:<br />
<style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Gray}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM-shift]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[gray]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: ({Gray}({x}) \wedge{} {g}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[[gray] cat]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: ({Gray}({x}) \wedge{} {Cat}({x}))$</div></div></td><td></td></tr></table><br /><br />Path [1]:<br />
<style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM-shift]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[cat]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$

### Quantifiers in object position

The "standard" approach to quantifiers in object position is to assume that a type mismatch can trigger QR.  However, an alternative is to type-shift the DP to a type where it can take a transitive predicate and ignore the external argument position.

This would need to be generalized for n-ary predicates.

In [10]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

INFO (meta): Coerced guessed type for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Human_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$<br />
$[\![\mathbf{\text{doctor}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Doctor}({x})$<br />
$[\![\mathbf{\text{someone}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({Human}({x}) \wedge{} {f}({x}))$<br />
$[\![\mathbf{\text{saw}}]\!]^{}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Saw}({y}, {x})$<br />
$[\![\mathbf{\text{alfonso}}]\!]^{}_{e} \:=\: $${Alfonso}_{e}$

In [11]:
((every * doctor) * (saw * alfonso)).tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{doctor}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Doctor}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[every doctor]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({Doctor}({x}) \rightarrow{} {g}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{saw}}]\!]^{}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Saw}({y}, {x})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{alfonso}}]\!]^{}_{e}$</div><div style="vertical-align:bottom;text-align:center">${Alfonso}_{e}$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[saw alfonso]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} y_{e} \: . \: {Saw}({y}, {Alfonso}_{e})$</div></div></td><td></td></tr></table></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span

In [12]:
(saw * (every * doctor))

CompositionResult(results=[], failures=[⟦[saw [every doctor]]⟧ = Type mismatch: '⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[every doctor] saw]⟧ = Type mismatch: '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Function Application), ⟦[saw [every doctor]]⟧ = Type mismatch: '⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[[every doctor] saw]⟧ = Type mismatch: '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Abstraction)])

The following combinator shifts a GQ type into something that can handle a transitive verb.

In [13]:
gq_lift_combinator = %te L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))
gq_lift_combinator

(λ f_<<e,t>,t>: (λ g_<e,<e,t>>: (λ x_e: f_<<e,t>,t>((λ y_e: g_<e,<e,t>>(y_e)(x_e))))))

In [14]:
gq_lift_combinator(someone.content).reduce_all()

(λ g_<e,<e,t>>: (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & g_<e,<e,t>>(x1_e)(x_e)))))

In [15]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [16]:
(alfonso * (saw * someone))

CompositionResult(results=[⟦[[[someone] saw] alfonso]⟧ = (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(Alfonso_e, x1_e)))], failures=[⟦[alfonso [[someone] saw]]⟧ = Type mismatch: '⟦alfonso⟧ = Alfonso_e'/e and '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Function Application), ⟦[alfonso [[someone] saw]]⟧ = Type mismatch: '⟦alfonso⟧ = Alfonso_e'/e and '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[alfonso [[someone] saw]]⟧ = Type mismatch: '⟦alfonso⟧ = Alfonso_e'/e and '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[someone] saw] alfonso]⟧ = Type mismatch: '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦alfonso⟧ = Alfonso_e'/e conflict (mode: Predicate Abstraction)])

In [17]:
(someone * (saw * (every * doctor)))

CompositionResult(results=[⟦[someone [[[every doctor]] saw]]⟧ = (Exists x_e: (Human_<e,t>(x_e) & (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e)))))], failures=[⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

In [18]:
r = ((every * doctor) * (saw * someone))
r

CompositionResult(results=[⟦[[every doctor] [[someone] saw]]⟧ = (Forall x_e: (Doctor_<e,t>(x_e) >> (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e)))))], failures=[⟦[[[someone] saw] [every doctor]]⟧ = Type mismatch: '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[every doctor] [[someone] saw]]⟧ = Type mismatch: '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[every doctor] [[someone] saw]]⟧ = Type mismatch: '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[someone] saw] [every doctor]]⟧ = Type mismatch: '⟦[[someone] saw]⟧ = (λ x_e: (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦[every doctor]⟧ = (λ g_<e,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

In [19]:
r.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{doctor}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Doctor}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[every doctor]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({Doctor}({x}) \rightarrow{} {g}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{someone}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({Human}({x}) \wedge{} {f}({x}))$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[gq-lift-trans]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[someone]}}]\!]^{}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \exists{} x1_{e} \: . \: ({Human}({x1}) \wedge{} {g}({x1})({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style

This so far produces only surface scope readings when there are multiple quantifiers.

Following work in CCG, one might imagine that composition needn't match constituency; if the subject shifts and composes with the verb before the object we can get the other scoping.  (In CCG this is implemented using a function composition operation, not a type-shift.)

In [20]:
r = (someone * (saw * (every * doctor)))
r

CompositionResult(results=[⟦[someone [[[every doctor]] saw]]⟧ = (Exists x_e: (Human_<e,t>(x_e) & (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e)))))], failures=[⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

Someone interested in constituency might find this unsatisfying.  How could this be resolved using a type-shift?  One idea (due to Hendriks) is to build scope-taking shifts that operate on verb meanings.  This is not necessarily more satisfying..

In [21]:
surface_shift_comb = te("L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : g(L y_e : f(L x_e : (v(x)(y))))")
inverse_shift_comb = te("L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : f(L x_e : g(L y_e : (v(x)(y))))")

inverse_shift_comb(saw.content).reduce_all()

(λ f_<<e,t>,t>: (λ g_<<e,t>,t>: f_<<e,t>,t>((λ x_e: g_<<e,t>,t>((λ y_e: Saw_<(e,e),t>(y_e, x_e)))))))

In [22]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(surface_shift_comb, "surface", typeshift=True))
system.add_rule(lang.unary_factory(inverse_shift_comb, "inverse", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [23]:
r = (someone * ((every * doctor) * saw))
r

CompositionResult(results=[⟦[[[saw] [every doctor]] someone]⟧ = (Exists x_e: (Human_<e,t>(x_e) & (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))), ⟦[[[saw] [every doctor]] someone]⟧ = (Forall x_e: (Doctor_<e,t>(x_e) >> (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x1_e, x_e)))))], failures=[⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: g_<<e,t>,t>((λ y_e: (Forall x_e: (Doctor_<e,t>(x_e) >> Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: g_<<e,t>,t>((λ y_e: (Forall x_e: (Doctor_<e,t>(x_e) >> Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Modification), ⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: g_<<e,t>,t>((λ y_e: (Forall x_e: (Doctor_<e,t>(x_e) >> Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Abstraction), ⟦[[[saw] [every doctor]] someone]⟧ = Type mismatch: '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: g_<<e,t>,t>((λ y_e: (Forall x_e: (Doctor_<e,t>(x_e) >> Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<<e,t>,t>((λ y_e: Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<<e,t>,t>((λ y_e: Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Modification), ⟦[someone [[saw] [every doctor]]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<<e,t>,t>((λ y_e: Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Abstraction), ⟦[[[saw] [every doctor]] someone]⟧ = Type mismatch: '⟦[[saw] [every doctor]]⟧ = (λ g_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> g_<<e,t>,t>((λ y_e: Saw_<(e,e),t>(y_e, x_e))))))'/<<<e,t>,t>,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

In [24]:
r[1].tree()

RecursiveDerivationDisplay instance: HTML rendering only

A final strategy would be to provide an even higher object type-lift that implements inverse scope.  This is effectively the combinator for Hendriks' inverse scope shifter with the order of arguments reversed.

In [25]:
gq_lift_combinator = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))")
gq_lift_combinator2 = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L h_<<e,t>,t> : f(L y_e : h(L x_e : g(y)(x)))")

gq_lift_combinator2.type

<<<e,t>,t>,<<e,<e,t>>,<<<e,t>,t>,t>>>

In [26]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.add_rule(lang.unary_factory(gq_lift_combinator2, "gq-lift2-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [27]:
r = (someone * ((every * doctor) * saw))
r

CompositionResult(results=[⟦[someone [[[every doctor]] saw]]⟧ = (Exists x_e: (Human_<e,t>(x_e) & (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))), ⟦[[[[every doctor]] saw] someone]⟧ = (Forall x_e: (Doctor_<e,t>(x_e) >> (Exists x1_e: (Human_<e,t>(x1_e) & Saw_<(e,e),t>(x1_e, x_e)))))], failures=[⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ x_e: (Forall x1_e: (Doctor_<e,t>(x1_e) >> Saw_<(e,e),t>(x_e, x1_e))))'/<e,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ h_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> h_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ h_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> h_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Modification), ⟦[someone [[[every doctor]] saw]]⟧ = Type mismatch: '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> and '⟦[[[every doctor]] saw]⟧ = (λ h_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> h_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,t>,t>,t> conflict (mode: Predicate Abstraction), ⟦[[[[every doctor]] saw] someone]⟧ = Type mismatch: '⟦[[[every doctor]] saw]⟧ = (λ h_<<e,t>,t>: (Forall x_e: (Doctor_<e,t>(x_e) >> h_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,t>,t>,t> and '⟦someone⟧ = (λ f_<e,t>: (Exists x_e: (Human_<e,t>(x_e) & f_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

In [28]:
r.tree()

2 composition paths:<br />
Path [0]:<br />
<style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{someone}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({Human}({x}) \wedge{} {f}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{doctor}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Doctor}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[every doctor]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({Doctor}({x}) \rightarrow{} {g}({x}))$</div></div></td><td></td></tr></table></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[gq-lift-trans]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[[every doctor]]}}]\!]^{}_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} x1_{e} \: . \: ({Doctor}({x1}) \rightarrow{} {g}({x1})({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padd

### Partee's type-shifts

(needs to be added)